In [1]:
# import os
# import sys
# sys.path.append(os.path.abspath('') + '/..')

# from data import ImplicitData

from flurs.data.entity import User, Item, Event
from flurs.recommender import UserKNNRecommender
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../datasets/ml1m_gte5.csv", sep='\t')
user_col = 'UserID'
item_col = 'ItemID'
data.head()

,UserID,ItemID,Timestamp
0,6040,593,956703954
1,6040,2019,956703977
2,6040,213,956704056
3,6040,3111,956704056
4,6040,912,956704191


In [3]:
data2=data.copy()

In [4]:
usermap = pd.Series(pd.unique( data[user_col] )).reset_index().set_index(0).to_dict()['index']
itemmap = pd.Series(pd.unique( data[item_col] )).reset_index().set_index(0).to_dict()['index']
# 3m 24.7s
# def mapfun(x, unique=pd.unique( data[item_col] )):
#     return np.where( unique == x )[0][0]
# remaping user/item names so that they are ordered from 0 to number of users/items
# Flurs need user/item IDs to be increasing order integers
# data2[item_col]=data2[item_col].map(mapfun)
# data2[user_col]=data2[user_col].map( lambda x: mapfun( x, unique=pd.unique( data[user_col] ) ) )

data2[user_col]=data2[user_col].map(usermap)
data2[item_col]=data2[item_col].map(itemmap)

data2 = data2.iloc[:, :2].values

In [5]:
# Initialize model
knn_RS = UserKNNRecommender(k=5)
knn_RS.initialize()

In [6]:
# For each interaction - register user, register item, register event
for u, i in data2[:1000]:
    # print(u,i)
    user = User(u)
    knn_RS.register(user)

    item = Item(i)
    knn_RS.register(item)

    event = Event(user, item)
    knn_RS.update(event)

/home/klismam/.virtualenvs/dissertacao/lib/python3.6/site-packages/flurs/model/user_knn.py:80: RuntimeWarning: invalid value encountered in sqrt
  self.S[ua, idx] = self.B[ua, idx] / (np.sqrt(self.C[ua, idx]) * np.sqrt(self.D[ua, idx]))


KeyboardInterrupt: 

In [ ]:
knn_RS.users[0]['known_items']

set()

In [ ]:
for i in knn_RS.users[0]['known_items']:
    print(i)

In [ ]:
knn_RS.recommend(User(0), np.unique( data2[:1000, 1] ) ) # np.unique( data2[:, 1] ).reshape(1,-1)
# => (sorted candidates, scores)

(array([516, 192, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
        173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
        186, 187, 188, 189, 190, 161, 160, 159, 143, 130, 131, 132, 133,
        134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 158, 145, 146,
        147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 191, 193,
        257, 194, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
        238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250,
        251, 252, 253, 254, 255, 226, 225, 224, 208, 195, 196, 197, 198,
        199, 200, 201, 202, 203, 204, 205, 206, 207, 209, 223, 210, 211,
        212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 129, 128,
        127, 126,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
         44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
         57,  58,  59,  60,  61,  32,  31,  30,  14,   1,   2,   3,   4,
          5,   6,   7,   8,   9,  10,  11,  12,  13

In [8]:
import os
import sys
sys.path.append(os.path.abspath('') + '/..')
from data import ImplicitData, getBucketsHoldouts
from plot_utils import lineplot_recallxholdout, recall_heatmap
from dataset_evaluation_utils import *
from recommenders_implicit import ISGD, RAISGD, RSISGD  # ISGD framework, BISGD,
from eval_implicit import EvaluateHoldouts, EvaluateAndStore # EvaluateAndStore para guardar estados do modelo e holdouts, a avaliação prequencial de ratings implicitos é opcional, , EvalHoldout

from flurs.data.entity import User, Item, Event
from flurs.recommender import UserKNNRecommender, BPRMFRecommender, SketchRecommender
import pandas as pd
import numpy as np

from datetime import datetime
import joblib
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

In [3]:
# importa dataset 'books playlists'
data = pd.read_csv('output/amazonbooks_dump/3rd_sampled_amazon_books.csv')
dataset_name = 'Amazon_Books'
user_col = 'user_id'
item_col = 'item_id'
# data.sort_values(by='timestamp', inplace=True)

In [4]:
buckets = joblib.load('output/amazonbooks_dump/3rd_sample_buckets.joblib')
holdouts = joblib.load('output/amazonbooks_dump/3rd_sample_holdouts.joblib')

In [5]:
# https://flurs.readthedocs.io/en/latest/reference.html#evaluation-utilities
import time

class EvaluateHoldouts_Flurs():
    def __init__(self, model, buckets, holdouts, data, user_col, item_col):
        self.model = model
        self.buckets = buckets
        self.holdouts = holdouts
        self.usermap = pd.Series(pd.unique( data[user_col] )).reset_index().set_index(0).to_dict()['index']
        self.itemmap = pd.Series(pd.unique( data[item_col] )).reset_index().set_index(0).to_dict()['index']
        self.max_users = data[user_col].nunique()
        self.max_items = data[item_col].nunique()
        self.metrics = ["Recall@N"]
        self.IncrementalTraining_time_record = {}
        self.EvaluateHoldouts_time_record = {}

    def train_evaluate(self, N_recommendations=20, exclude_known_items:bool=True):
        '''
        Incremental training of recommendation model.
        '''
        # model_checkpoints = []
        self.results_matrix = np.zeros( shape=( len( self.holdouts ), len( self.holdouts ) ) )
        # metric = self.metrics[0]
        self.N_recommendations = N_recommendations
        self.exclude_known_items = exclude_known_items
        # Initialize model
        # model = UserKNNRecommender(k=5) # model
        self.model.initialize()
        print('register users')
        s = time.time()
        for u in range(0, self.max_users):
            if not ((u*100)/self.max_users)%5:
                print (((u*100)/self.max_users),'%')
            self.model.register(User(u))
        f = time.time()
        print( f-s, 'register users time' )
        print('register items')
        s = time.time()
        for i in range(0, self.max_items):
            if not ((i*100)/self.max_items)%5:
                print (((i*100)/self.max_items),'%')
            self.model.register(Item(i))
        f = time.time()
        print( f-s, 'register items time' )
        self.max_item_ID = 0
        self.cold_start_buckets = len( self.buckets ) - len( self.holdouts ) 
        # for each bucket, perform incremental training, evaluate model state against all holdouts
        for b, bucket in enumerate(self.buckets):
            print(f'Training: bucket {b}')
            incrtrain_time = []
            for i in range(bucket.size):
                if not ( (i*100)/bucket.size ) % 5:
                    print(( (i*100)/bucket.size ), '%')
                uid,iid = bucket.GetTuple(i)
                s = time.time()
                u_flurs, i_flurs = self.usermap[uid], self.itemmap[iid]
                self.max_item_ID = max(self.max_item_ID, i_flurs)
                # For each interaction - register user, register item, register event
                user = User(u_flurs)
                item = Item(i_flurs)
                event = Event(user, item)
#                 self.model.register(user)
#                 self.model.register(item)
                self.model.update(event)
                f = time.time()
                incrtrain_time.append(f-s)        

            self.IncrementalTraining_time_record[f'bucket_{b}'] = {
                'size':bucket.size,
                'train time vector':incrtrain_time,
                'avg train time':np.mean(incrtrain_time),
                'total train time':np.sum(incrtrain_time)
                }            

            if b >= self.cold_start_buckets:
                self._evaluate(n_bucket=b)

    def _evaluate(self, n_bucket):
        # after learning a bucket, evaluate model over every holdout     
        evaluate_time = [] 
        time_get_tuple = []
        time_recommend = []
        time_del_user_items = []
        time_eval_point = []  
        for i, hd in enumerate(self.holdouts):
            print(f'Evaluating: model {n_bucket-self.cold_start_buckets} x holdout {i}')
            start_evaluate_time = time.time()
            results = {}
            # for metric in self.metrics:
            metric = self.metrics[0]
            results[metric] = []
            for hd_i in range(hd.size):
                if not ( (hd_i*100)/hd.size ) % 5:
                    print(( (hd_i*100)/hd.size ), '%')
                # GetTuple
                s = time.time()
                uid, iid = hd.GetTuple(hd_i) # get external IDs
                u_flurs, i_flurs = self.usermap[uid], self.itemmap[iid]
                user = User(u_flurs)
                f = time.time()
                time_get_tuple.append(f - s)
                # recommend
                s = time.time()
                reclist = self.model.recommend(user, np.arange(self.max_item_ID+1) )[0]
                f = time.time()
                time_recommend.append(f - s)
                # get items seen by user up until bucket 'n_bucket'
                # delete seen items 
                s = time.time()
                user_items = self.buckets[n_bucket].GetUserItems(uid, internal=False) 
                for previous_b in range(n_bucket):
                    user_items = np.concatenate((user_items, self.buckets[previous_b].GetUserItems(uid, internal=False)))
                user_items = [self.itemmap[i] for i in user_items]
                if self.exclude_known_items:
                    reclist = np.delete(reclist, user_items)
                f = time.time()
                time_del_user_items.append(f - s)
                # get n recommendations
                n = self.N_recommendations
                if n == -1:
                    n = len(reclist)
                reclist = reclist[:n]
                if len(reclist):
                    s = time.time()
                    results[metric].append(self.__EvalPoint(i_flurs, reclist))
                    f = time.time()
                    time_eval_point.append(f - s)
                else:
                    print(uid, 'user not seen')
            result = results[metric]
            n_not_seen = hd.size - len(result) # if user was not seen, its not added to recall. May be needed to store difference.
            if n_not_seen:
                print(f'recommendations not made for users in holdout {i} x bucket {n_bucket}: {n_not_seen}')
            result = sum( result ) / len(result)
            self.results_matrix[n_bucket-self.cold_start_buckets, i] = result
            end_evaluate_time = time.time()
            evaluate_time.append(end_evaluate_time - start_evaluate_time)
            self.EvaluateHoldouts_time_record[f'model {n_bucket-self.cold_start_buckets} x holdout {i}'] = {
                'size': hd.size,
                'train time vector':evaluate_time,
                'avg model eval time':np.mean(evaluate_time),
                'total train time':np.sum(evaluate_time),
                'time_get_tuple': time_get_tuple,
                'time_recommend': time_recommend,
                'time_del_user_items': time_del_user_items,
                'time_eval_point': time_eval_point,
            }

    def __EvalPoint(self, item_id, reclist):
        result = 0
        if len(reclist) == 0:
            return 0
        for metric in self.metrics:
            if metric == "Recall@N":
                result = int(item_id in reclist)
        return result

In [7]:
eval_KNN = EvaluateHoldouts_Flurs(UserKNNRecommender(k=5), buckets, holdouts, data, user_col, item_col)

In [ ]:
%%time
# quase 3 horas só pra registrar usuários e itens
eval_KNN.train_evaluate(N_recommendations = 20, exclude_known_items = True)

register users
0.0 %
20.0 %
40.0 %
60.0 %
80.0 %
0.2770051956176758 register users time
register items
0.0 %
20.0 %
40.0 %


In [9]:
BPRMF = EvaluateHoldouts_Flurs(BPRMFRecommender(), buckets, holdouts, data, user_col, item_col)

In [10]:
%%time
# bem mais rápido registrar usuários e itens com BPRMF do que com KNN
BPRMF.train_evaluate(N_recommendations = 20, exclude_known_items = True)

register users
0.0 %
20.0 %
40.0 %
60.0 %
80.0 %
0.010806798934936523 register users time
register items
0.0 %
20.0 %
40.0 %


KeyboardInterrupt: 

In [ ]:
eval_KNN

In [ ]:
import json
json.dump()